### Custom operator

Custom operator allows developers to override or run un-supported operators with their custom implementation in swift.

#### When shall I need to use Custom operators?
1. My TensorFlow/PyTorch model has an operator which is not supported by converter
2. My use-case is special and require different behavior compared to default one

#### Composite operators and Custom operators
- Composite operators are subset of Custom operators
- Composite operators are frontend specific and uses CoreML builder APIs to 
  inserts runtime supported operators
- Whereas, Custom operators allows one to write their own implementation that will
  be invoked during runtime

#### When to use Custom operator over Composite operator
- Operator is not supported by runtime
- Can provide efficient implementation and would like to benefit from it
- Operator is complex and difficult to implement with CoreML builder API
  e.g. Convolution, LSTM
- Implementing via Composite operator is not performant
  e.g. with custom implementation can have one operator instead of with
  Composite operator having multiple operator and can write an effiencient
  implementation

In [1]:
import coremltools
import tensorflow as tf
import coremltools.converters.nnv2.converter as converter
from coremltools.models.neural_network.printer import print_network_spec
from _tf_utils import *

In [ ]:
from packaging import version
assert version.parse(tf.__version__).major < 2, "Following tutorial does not work for TF 2.0"

In [2]:
from coremltools.converters.nnv2.frontend.tensorflow.tf_op_registry import register_tf_op
from coremltools.converters.nnv2.nnv2_program.ops import CoremlBuilder as cb
from coremltools.converters.nnv2.nnv2_program.ops.defs._op_reqs import *
from coremltools.converters.nnv2.builtin_types.symbolic import is_symbolic

In [3]:
# If custom implementation is required
# Following specification is required to bind custom operator to custom layer
# implementation

# What all should be specified in following specification
# 1. is_custom_op should be set to True while registering op
# 2. bindings should be provided as part of the operator class
#    with following information:
#    a) class_name: Name of the class (Interface name of custom layer implementation)
#    b) input_order: Inpur order from above named input being used in custom implementation.
#                    Inputs will be packed as a List of Multi-Array and pass in this order.
#    c) parameters: Parameters that should be passed as operator attributes and known.
#    d) description: Short description of current operator

# Defining SSA TopK Op
@register_op(doc_str='Custom TopK Layer', is_custom_op=True)
class custom_topk(Operation):
    input_spec = InputSpec(
             x = TensorInputType(),
             k = IntInputType(const=True, default=1),
          axis = IntInputType(const=True, default=-1),
        sorted = BoolInputType(const=True, default=False),
    )

    bindings = { 'class_name'  : 'CustomTopK',
                 'input_order' : ['x'],
                 'parameters'  : ['k', 'axis', 'sorted'],
                 'description' : "Top K Custom layer"
                }

    def __init__(self, **kwargs):
        super(custom_topk, self).__init__(**kwargs)

    def type_inference(self):
        x_type = self.x.dtype
        x_shape = self.x.shape
        k = self.k.val
        axis = self.axis.val

        if not is_symbolic(x_shape[axis]) and k > x_shape[axis]:
            msg = 'K={} is greater than size of the given axis={}'
            raise ValueError(msg.format(k, axis))

        ret_shape = list(x_shape)
        ret_shape[axis] = k
        return builtins.tensor(x_type, ret_shape), builtins.tensor(builtins.int32, ret_shape)

In [4]:
# Override TopK op with override=True flag
@register_tf_op(tf_alias=['TopKV2'], override=True)
def CustomTopK(context, node):
    x = context[node.inputs[0]]
    k = context[node.inputs[1]]
    sorted = node.attr.get('sorted', False)
    x = cb.custom_topk(x=x, k=k.val, axis=-1, sorted=sorted, name=node.name)
    context.add(node.name, x)

In [5]:
def test_custom_topk_op_conversion(rank, k):
    shape = np.random.randint(low=3, high=6, size=rank)
    input = np.random.rand(*shape)
    with tf.Graph().as_default() as graph:
        x = tf.placeholder(tf.float32, shape=shape)
        ref = tf.math.top_k(x, k=k, sorted=True)
        ref = (ref[0], ref[1])

        spec = convert_tf1(graph, {x: input}, ref)
        # Validate custom layer is added
        layers = spec.neuralNetwork.layers
        assert layers[-1].custom is not None, "Expecting a custom layer"
        assert 'CustomTopK' == layers[-1].custom.className, "Custom Layer class name mis-match"
        assert k == layers[-1].custom.parameters['k'].intValue, "Incorrect parameter value k"
        assert True == layers[-1].custom.parameters['sorted'].boolValue, "Incorrect parameter value for Sorted"
        print("Model converted successfully with custom layer `CustomTopK`")
        print("Please make sure Swift/Objective-C implemenetation of `CustomTopK` is in scope while building your app!!\n")
        print("Following you converted model:\n")
        print_network_spec(spec, style='coding')

In [6]:
test_custom_topk_op_conversion(4, 2)

Model converted successfully with custom layer `CustomTopK`
Please make sure Swift/Objective-C implemenetation of `CustomTopK` is in scope while building your app!!

Following you converted model:

Inputs:
  Placeholder [5, 5, 4, 3]
Outputs:
  TopKV2:0 []
  TopKV2:1 []


def model(Placeholder):
	TopKV2:0, TopKV2:1 = custom (Placeholder)
 
	return TopKV2:0, TopKV2:1
